In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import array
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

In [3]:
data =  pd.read_csv('GOTO.JK.csv')
data = data['Volume']
data.dropna(inplace=True)

In [4]:
n = len(data)
sizeTrain = (round(n*0.8))
data_train = pd.DataFrame(data[:sizeTrain])
data_test = pd.DataFrame(data[sizeTrain:])

In [5]:
# data_training, data_testing = train_test_split(data,train_size=0.8, test_size=0.2, random_state=42, shuffle=False)
# data_training

In [6]:
# data_training_new = pd.DataFrame(data_training)
# data_test_new = pd.DataFrame(data_testing)
# data_training_new

In [72]:
from sklearn.decomposition import PCA

# Membuat objek PCA dengan jumlah komponen yang diinginkan
pca = PCA(n_components=1)

# Melakukan reduksi dimensi pada data
reduced_data = pca.fit_transform(data_train)
datapca = pd.DataFrame(reduced_data)
datapca

,0
0,1.427678e+09
1,1.797116e+09
2,5.523681e+08
3,2.047174e+09
4,9.293073e+07
...,...
193,8.666235e+08
194,7.640793e+08
195,5.583674e+10
196,4.699331e+08


In [57]:
import joblib
filename = 'ReDim.pkl'
joblib.dump(datapca, filename) 

['ReDim.pkl']

In [50]:
Mm = MinMaxScaler()
train_scaled = Mm.fit_transform(data_train)
test_scaled = Mm.transform(data_test)
TRAIN = pd.DataFrame(train_scaled)
TRAIN

,0
0,0.059662
1,0.066047
2,0.044535
3,0.070369
4,0.036594
...,...
193,0.049966
194,0.048194
195,1.000000
196,0.043110


In [51]:
import joblib
filename = 'MinMax.pkl'
joblib.dump(TRAIN, filename) 

['MinMax.pkl']

In [8]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [9]:
n_steps = range(1,4)
acc_fix = {}
for n in n_steps:
    X_train, y_train = split_sequence(train_scaled, n)
    X_test, y_test = split_sequence(test_scaled, n)
    #membuat kolom otomatis
    colom = []
    for c in range(n):
        if(c==0):
            colom.insert(0,'Xt')
        else:
            colom.insert(0,f'Xt-{c}')
    newX_train = pd.DataFrame(np.reshape(X_train,(len(X_train),n)),columns=colom)
    newY_train = pd.DataFrame(y_train, columns=['Output'])
    new_data_train = pd.concat([newX_train,newY_train],axis=1)

    newX_test = pd.DataFrame(np.reshape(X_test,(len(X_test),n)),columns=colom)
    newY_test = pd.DataFrame(y_test, columns=['Output'])
    new_data_test = pd.concat([newX_test,newY_test],axis=1)

    #membuat model
    k_nei = range(1,30)
    acc = []
    for k in k_nei:
        neigh = KNeighborsRegressor(n_neighbors=k)
        neigh.fit(newX_train, newY_train)
        y_pred = neigh.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = Mm.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = Mm.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc.append(mape)
    acc_fix[f"n-{n} knn"] = [min(acc), acc.index(min(acc))]
    # predict gaussian regression
    karnel = DotProduct() + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=karnel,random_state=0).fit(newX_train, newY_train)
    y_pred = gpr.predict(newX_test, return_std=True)[0]
    y_pred_shape = y_pred.reshape(-1,1)
    y_pred_inverse = Mm.inverse_transform(y_pred_shape)
    newY_test_shape = y_test.reshape(-1,1)
    newY_test_inverse = Mm.inverse_transform(newY_test_shape)
    # print(y_pred_inverse)
    mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)

    acc_fix[f"n-{n} gpr"] = [mape]
# predict SVR dengan Bagging Regression
    n_estimators = range(1, 30)
    acc_svr = []
    for n in n_estimators:
        regr = BaggingRegressor(SVR(), n_estimators=n,random_state=0).fit(newX_train, newY_train.values.ravel())
        # regr.fit(newX_train, newY_train)
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = Mm.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = Mm.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(newY_test_inverse, y_pred_inverse)
        acc_svr.append(mape)

    acc_fix[f"n-{n} SVR"] = [min(acc_svr), acc_svr.index(min(acc_svr))]
    # Predict dengan from RandomForestRegressor
    maxd = range(1,10)
    acc_rf = []
    for d in maxd:
        rfr = RandomForestRegressor(max_depth=d, random_state=0).fit(newX_train, newY_train.values.ravel())
        y_pred = regr.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = Mm.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = Mm.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_rf.append(mape)
    
    # masukkan ke acc-fix
    acc_fix[f"n-{n} rf"] = [min(acc_rf) , acc_rf.index(min(acc_rf))]

    # Prdict dengan LinearSVR , Ridge pada stacking regressors
    estimators = [('lr', RidgeCV()),
                ('svr', LinearSVR(random_state=42))]
    acc_stac = []
    for e in n_estimators:
        reg = StackingRegressor(
            estimators,
            final_estimator=RandomForestRegressor(n_estimators=e,
                                                random_state=42))
        reg.fit(newX_train, newY_train.values.ravel())
        y_pred = reg.predict(newX_test)
        y_pred_shape = y_pred.reshape(-1,1)
        y_pred_inverse = Mm.inverse_transform(y_pred_shape)
        newY_test_shape = y_test.reshape(-1,1)
        newY_test_inverse = Mm.inverse_transform(newY_test_shape)
        mape = mean_absolute_percentage_error(y_pred_inverse, newY_test_inverse)
        acc_stac.append(mape)
    acc_fix[f"n-{n} svr"] = [min(acc_stac), acc_stac.index(min(acc_stac))]

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

In [13]:
acc_fix

{'n-1 knn': [0.7749633475754272, 27],
 'n-1 gpr': [1.2841977756030616],
 'n-29 SVR': [1.1923528146333733, 6],
 'n-29 rf': [0.6192408233140794, 0],
 'n-29 svr': [0.8913644647166626, 15],
 'n-2 knn': [0.771207970758184, 28],
 'n-2 gpr': [1.1215533785513536],
 'n-3 knn': [0.6578059573801174, 21],
 'n-3 gpr': [0.9796458627027838]}

In [ ]:
import joblib
filename = 'Mm.pkl'
joblib.dump(Mm, filename) 

['Mm.pkl']

In [ ]:
import joblib
filename = 'RandomForest.pkl'
joblib.dump(rfr, filename) 

['RandomForest.pkl']

In [45]:
temp = 1
temp2 = {}
for m in acc_fix:
    if(acc_fix[m][0] < temp ):
        temp = acc_fix[m][0]
        temp2 = {}
        temp2[m] = acc_fix[m]


print(f"Hasil Dari Berbagai Model yang memiliki MAPE terkecil adalah: {temp2}")
knn = acc_fix['n-1 knn']
rf = acc_fix['n-29 rf']
svr = acc_fix['n-29 svr']
# knn.to_csv('knn.txt', index=False)
# acc_fix[m][0]
# acc_fix[m]
# temp2
# m

Hasil Dari Berbagai Model yang memiliki MAPE terkecil adalah: {'n-29 rf': [0.6192408233140794, 0]}


In [38]:
import joblib
filename = 'knn.pkl'
joblib.dump(knn, filename) 

['knn.pkl']

In [47]:
import joblib
filename = 'RandomForest.pkl'
joblib.dump(rf, filename) 

['RandomForest.pkl']

In [48]:
import joblib
filename = 'BaggingRegressor.pkl'
joblib.dump(svr, filename) 

['BaggingRegressor.pkl']

In [67]:
# create list 1 to 10
list = []
for i in range(1,11):
    list.append(i)
    
    # print()
